# Facebook Prophet

### Обучение модели Facebook

install packages

In [2]:
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
import math
import pandas.tseries.offsets as ofs
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa import stattools

import warnings as w
import plotly as py

%matplotlib inline
from fbprophet import Prophet
import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'fbprophet'

Загрузка данных

In [ ]:
ts= pd.read_csv('dataset1_with_outliers_data.csv')
ts.info

In [ ]:
ts.head()

In [ ]:
ts.head(950)

Видем, что есть интервал с пропущенными значениями. Перенесем их в отдельную переменную.

In [ ]:
ts.index = pd.to_datetime(ts['ds'])

ts = pd.read_csv('dataset1_with_outliers_data.csv')
ts.index = pd.to_datetime(ts['ds'])
ts.sort_index(inplace=True)

ts_test = ts[datetime(2015,1,1):]

ts_full = ts[:datetime(2015,1,1)]
ts_no_outs = ts_full[datetime(2012, 1,1):]

Посмотрим на весь график :

In [ ]:
ts_full.plot(figsize=(15,5))

И data_wihout_outs

In [ ]:
ts_no_outs.plot(figsize=(15,5))

### Проверим ряд на стационарность :

Проверьте ряд на стационарность (например, с помощью критерия Дики-Фуллера). Попробуйте привести его к стационарному виду (с помощью преобразования Бокса-Кокса, дифференцирования etc.)
После получения стационарного ряда напишите функцию прямой transform и обратной inv_transform трансформации временного ряда (т.е. исходный ряд -> стационарный ряд и стационарный ряд -> исходный ряд).

In [ ]:
print(" Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(ts_no_outs['y'])[1])

In [ ]:
def transform(df):
    plt.figure(figsize=(15,10))
    sm.tsa.seasonal_decompose(df['y'], freq = 30).plot()
    print(" Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['y'])[1])

### Попробуем обноружить выбросы :

С помощью экспоненциального сглаживания найдите выбросы в данных (и попробуйте их сгладить). Для этого подберите оптимальные значения параметров alpha, beta и std_window. Функция для детекции аномалий и сглаживания приведена ниже. Обратите внимание, мы работаем лишь с ts_train, тестовый датасет ts_test "находится в будущем" и нам недоступен.

In [ ]:
def exponential_smoothing_anomaly_filter(series, alpha, beta=2.5, std_window=10):



    result = [series[0]] # first value is same as series
    sx = series.rolling(std_window).std()
    anomalies_index = []
    anomalies_values = []
#     print(len(series), len(sx))
    for n in range(1, len(series)):
        if n > std_window:
            if abs(series[n] - result[n-1]) > sx[n]*beta:
                result.append(result[n-1])
                anomalies_index.append(series.index[n])
                anomalies_values.append(series[n])
                continue
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return pd.Series(index=series.index, data=result), pd.Series(index=anomalies_index, data=anomalies_values)

In [ ]:
ts_no_outs['ty'], ts_no_outs['ay'] = exponential_smoothing_anomaly_filter(ts_no_outs['y'], 0.35, 1.5, 29)
dk = sm.tsa.stattools.adfuller(ts_no_outs['ty'])[1]
dk

In [ ]:
plt.figure(figsize=(25,10))
ts_no_outs['ty'].plot()
print(" Критерий Дики-Фуллера: p=%f" % dk)

### Построем прогнозы с помощью библиотеки Facebook Prophet

In [ ]:
from fbprophet import Prophet
import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore")
fmodel = Prophet()


fmodel.fit(ts_no_outs)
future = fmodel.make_future_dataframe(periods=365)

forecast = fmodel.predict(future)

In [ ]:
fmodel.plot(forecast)

In [ ]:
fmodel.plot_components(forecast)

# ts_full

In [ ]:
fmodel1 = Prophet()

fmodel1.fit(ts_full)
future1 = fmodel1.make_future_dataframe(periods=365)

forecast1 = fmodel1.predict(future1)

In [ ]:
fmodel1.plot(forecast1)

Оценим качество прогноза с помощью MSE, MAE и $r^2$-score.

In [ ]:
# from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [ ]:
cmp_df  = make_comparison_dataframe(ts_no_outs, forecast)

In [ ]:
import numpy as np
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MSE {}'.format(np.mean(abs(cmp_df[datetime(2015,1,1):]['p']))))
print('MAE {}'.format(np.mean(abs(cmp_df[datetime(2015,1,1):]['e']))))

In [ ]:
fmodel.plot(forecast)

In [ ]:
fmodel1.plot(forecast1)

### Автоматическая работа с пропущенными значениями (25%)

На самом деле, библиотека Facebook Prophet умеет работать с пропущенными значениями. Создадим копию ts_copy ряда ts_full и заменим в ней значения с 2010-01-01 по 2011-01-01 на None.

In [ ]:
ts_copy = ts_full.copy()
ts_copy['y'][datetime(2010,1,1):datetime(2011,1,1)] = None
ts_copy[datetime(2010,1,1):datetime(2011,1,1)]

In [ ]:
fmodel2 = Prophet()

fmodel2.fit(ts_copy)
future2 = fmodel2.make_future_dataframe(periods=365)

forecast2 = fmodel2.predict(future2)

In [ ]:
fmodel2.plot(forecast2)

In [ ]:
cmp_df  = make_comparison_dataframe(ts_copy, forecast2)

In [ ]:
import numpy as np
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MSE {}'.format(np.mean(abs(cmp_df[datetime(2015,1,1):]['p']))))
print('MAE {}'.format(np.mean(abs(cmp_df[datetime(2015,1,1):]['e']))))

MSE было 9.171853243470927, а стало 11.15781712755453, алгоритм хуже предсказывает.
Добавим информации о выходных днях(суббота и воскресенье)

In [ ]:
saturday  = pd.DataFrame({
  'holiday': 'saturday',
  'ds': ts.loc[ts.index.weekday == 5].ds
})


sunday  = pd.DataFrame({
  'holiday': 'sunday',
  'ds': ts.loc[ts.index.weekday == 6].ds
})

In [ ]:
fmodel2.plot_components(forecast2)

Построем прогноз на год вперед c учетом выходных дней на основании ts_copy

In [ ]:
ts_copy = ts_full.copy()
ts_copy['y'][datetime(2011,1,1):datetime(2012,1,1)] = None
ts_copy[datetime(2011,1,1):datetime(2012,1,1)]

In [ ]:
fmodel4 = Prophet()
fmodel4.fit(ts_copy)
future4 = fmodel2.make_future_dataframe(periods=365)
forecast4 = fmodel4.predict(future4)

In [ ]:
fmodel4.plot(forecast4)

Оценим качество прогноза с помощью MSE, MAE и $r^2$-score еще раз.

In [ ]:
import numpy as np
cmp_df  = make_comparison_dataframe(ts_copy, forecast4)
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MSE {}'.format(np.mean(abs(cmp_df[datetime(2015,1,1):]['p']))))
print('MAE {}'.format(np.mean(abs(cmp_df[datetime(2015,1,1):]['e']))))

MSE было 11.15781712755453, а стало 11.227507141252778, cредняя ошибка возросла при пропущенных данных, следовательно алгоритм стал хуже предсказывать.